Import Packages 

In [3]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import os
import scipy.misc as scipyimg
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib inline

Lane Finder Using Canny Edge Detector

In [6]:
#Takes the name of an image as input and saves the augmented image in the test_images_output folder
def process_image(image):
    #Convert to grayscale
    gray = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)

    #Do gaussian smoothing
    kernel_size = 3
    blur_gray = cv2.GaussianBlur(gray,(kernel_size,kernel_size),0)

    #Run Canny edge detection
    low_threshold = 50
    high_threshold = 100
    edges = cv2.Canny(blur_gray, low_threshold, high_threshold)

    # Define Region of Interest and mask edges outside of it
    mask = np.zeros_like(edges)   
    ignore_mask_color = 255   

    # Define region using a 4 sided polygon
    imshape = image.shape
    vertices = np.array([[(250,imshape[0]-60),(480, 460), (imshape[1]-440, 460), (imshape[1]-150,imshape[0]-60)]], dtype=np.int32)
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    masked_edges = cv2.bitwise_and(edges, mask)

    #plt.imshow(masked_edges)


    #Define Hough Transform Parameters
    rho = 1 # distance resolution in pixels of Hough grid
    theta = np.pi/180 # angular resolution in radians of grid
    threshold = 20 #miminum number of votes or intersections in hough grid
    min_line_length = 30 #minimum number of pixels making up a line
    max_line_gap = 5 #maximum gap in pixels between line segments
    line_image = np.zeros_like(image)

    #Run hough transform
    lines = cv2.HoughLinesP(masked_edges,
                            rho, theta, threshold, np.array([]), min_line_length, max_line_gap)

    #Collection of x and y points in left lane and right lane
    left_lane = [[],[]]
    right_lane = [[],[]]

    for line in lines:
        for x1,y1,x2,y2 in line:
            if(((y2-y1)/(x2-x1) > 0.5) and (x2 > 480 and x1 > 480)):
                right_lane[0].extend([x1,x2])
                right_lane[1].extend([y1,y2])
                cv2.line(line_image,(x1,y1),(x2,y2),(0,0,255),6)
            elif (((y2-y1)/(x2-x1) < -0.5) and (x2 < 480 and x1 < 480)):
                left_lane[0].extend([x1,x2])
                left_lane[1].extend([y1,y2])
                cv2.line(line_image,(x1,y1),(x2,y2),(0,0,255),6)

    #If points for left lane are found, extrapolate lane and draw onto image
    if(len(left_lane[0]) != 0):
        left_line = np.polyfit(left_lane[0],left_lane[1],1)
        y1 = imshape[1]
        x1 = int((y1-left_line[1])/left_line[0])
        y2 = 440 #height we want the projected lines to go to
        x2 = int((y2-left_line[1])/left_line[0])
        cv2.line(line_image, (x1, y1), (x2, y2), [255,0,0], 8)
        #cv2.line(line_image, (250,imshape[0]-100), (450, 460), [255,0,0], 8)
    
    #If points for right lane are found, extrapolate lane and draw onto image
    if(len(right_lane[0])!= 0):
        right_line = np.polyfit(right_lane[0],right_lane[1],1)
        y1 = imshape[1]
        x1 = int((y1-right_line[1])/right_line[0])
        y2 = 440 #height we want the projected lines to go to
        x2 = int((y2-right_line[1])/right_line[0])
        cv2.line(line_image, (x1, y1), (x2, y2), [255,0,0], 8)
        #cv2.line(line_image, (imshape[1]-400, 460), (imshape[1]-150,imshape[0]-100), [255,0,0], 8)

    #create a color binary image to combine with the line image
    #color_edges = np.dstack((edges, edges, edges))

    #Draw the lines on the edge image
    augmented_lanes = cv2.addWeighted(image, 0.8, line_image, 1, 0)

    #scipyimg.imsave("test_images_output/" + image_name, augmented_lanes)
    return augmented_lanes

Reading Images

In [65]:
#reading in an image
images = os.listdir("test_images/")
for image_name in images:
    image = mpimg.imread("test_images/" + image_name)
    augment_lanes(image)

Processing Video

In [7]:
challenge_output = 'test_videos_output/challenge.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip3 = VideoFileClip('test_videos/challenge.mp4').subclip(0,5)
clip3 = VideoFileClip('test_videos/challenge.mp4')
challenge_clip = clip3.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/challenge.mp4
[MoviePy] Writing video test_videos_output/challenge.mp4


100%|██████████| 251/251 [00:30<00:00,  5.40it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/challenge.mp4 

CPU times: user 1min 11s, sys: 1.25 s, total: 1min 13s
Wall time: 33.3 s
